## Setup

In [8]:
from functions import Model, get99, splitData, dropMissingValues, z_score, outliers_knn, outliers_dbscan, getNoise, getRelFeatures, drop_features, getCombinations, get_unique_list, bold, blue, red, green, getBestModel, np_to_df, df_to_np, drop99_all, z_score_individual
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
import json
import numpy as np
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [9]:
# JSON file
f = open ('data/json/input.json', "r")
  # Reading from file
PARAMETERS = json.loads(f.read())

SPLIT_SIZE = PARAMETERS["SPLIT_SIZE"]

OUTLIER99 = PARAMETERS["OUTLIER99"] 

OUTLIER_ZSCORE_IND = PARAMETERS["OUTLIER_ZSCORE_IND"]

OUTLIER_ZSCORE = {"STD" : [10.1, 10.2, 10.3, 10.4]}
OUTLIER_KNN = {"K": [3], "X": [181, 300, 500 , 700, 1000, 2000]}
OUTLIER_DBSCAN = {"K": [3], "X": [181, 2000], "EPS": [0.3, 0.7], "MIN_SAMPLES":  [3, 9, 10]}
OUTLIER_NOISE = {"CV": [3,4,5,6,7,8,9,10]}

FEATURE_SEL_CORR = {"THRESHOLD_CORR": [0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]}

f.close()

#TODO FEATURE
#TODO Regressionmodels
#TODO TESTDATA --> path to csv-file


In [10]:
#Get raw Data
df = pd.read_csv("data/csv/house_data_training.csv", sep=';') 
# remove unnamed column
df = df.iloc[:, 1:]
#Transform string to datetime
df["date"] = pd.to_datetime(df["date"])

maeList = []#for model selection

In [11]:
df = dropMissingValues(df) 

## Outlier Detection

### ZScore

In [12]:
# try:
#     # Outlier Detection
#     outlier_dict_all = {}

#     if OUTLIER99 == True:
#         print("outlier_99")
#         outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
#         df = drop99_all(df, outlier_list_99 )
#         #outlier_dict_all["99_mean"] = outlier_list_99
#         #outlier_dict_all["99_drop"] = outlier_list_99
#         print("done\n")

#     print('starting z_score')
#     for p in OUTLIER_ZSCORE["STD"]:
#         print(p)
#         outlier_list_z_score = z_score(df, p)
#         outlier_dict_all["z_score_"+str(p)] = outlier_list_z_score

#     ''''''
#     outlier_lists_all = list(outlier_dict_all.values())
#     outlier_lists_keys = list(outlier_dict_all.keys())
#     combination_list = []
#     for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
#         if not i == 1: #emtpy tuple
#             combination_list.append(combo)
#             # print('combo #{}: {}'.format(i, combo))
#     #print("...")
#     #for c in combination_list[10:]:
#         #print(bold("Combination: "), blue(c),"\n",)
#     #print(bold(len(combination_list)), " total Combinations")

#     model_obj_list = []
#     for i, c in enumerate(combination_list):
#         #print(f'\n{bold(i)}: {blue(c)}')
#         outlier_list = get_unique_list(outlier_dict_all, c)
#         try: 
#             if "99_mean" in c:
#                 X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
#             else:
#                 X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")

#             #TODO Regressionmodels
#             model = LinearRegression()
#             obj = Model(model, (X_train, X_test, y_train, y_test), c, df)
#             model_obj_list.append(obj)

#         except Exception as e:
#             print(bold("Error"), red(e))

#     #Summary
#     df_summary = pd.DataFrame(columns=['combo', 'mae', 'score'])
#     for o in model_obj_list:
#         #print(f'{o.get_type()}, {o.get_comb()}')
#         o.train()
#         mae, score = o.summary()
#         comb = o.get_comb()
#         row = {'combo': comb, 'mae': mae, 'score': score}
#         df_summary = df_summary.append(row, ignore_index=True)

#     df_summary.sort_values("mae").head()

#     ##row
#     mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
#     score_best = df_summary[df_summary.score==df_summary.score.max()]
#     best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
#     print(best_model_obj)
#     #print(f'{bold("Lowest mae:")} {mae_best}')

#     model_obj_list_best = [best_model_obj]
#     df_feature_reduction = best_model_obj.get_df().head()

#     # print(df_feature_reduction)

#     ###Feature
#     for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
#         #print(threshold, type(threshold))
#         try: 
#             rel_features = getRelFeatures(df_feature_reduction, threshold)
#             #print(len(rel_features), rel_features)
#             df_feature_reduction = df_feature_reduction[rel_features]
#             X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
#             best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction)
#             # best_model_obj_new.train()
#             model_obj_list_best.append(best_model_obj_new)

#         except Exception as e:
#             print(bold("Error"), red(e))
#     #print(model_obj_list_best)

#     #Summary
#     df_summary_best = pd.DataFrame(columns=['combo', 'mae', 'score', 'features'])
#     for o in model_obj_list_best:
#         #print(f'{o.get_type()}, {o.get_comb()}')
#         o.train()
#         mae, score = o.summary()
#         comb = o.get_comb()
#         features = o.get_df().columns.to_list()
#         features.remove("price")
#         row = {'combo': comb, 'mae': mae, 'score': score, 'features': features}
#         df_summary_best = df_summary_best.append(row, ignore_index=True)

#     #df_summary_best.sort_values("mae")

#     #row
#     mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
#     score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
#     best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
#     print(best_model_obj)
#     print(f'{bold("Lowest mae:")} {mae_best}')

#     lines = f'{df_summary_best["combo"][0]} - {df_summary_best["mae"][0]} - {len(df_summary_best["features"][0])} ' 

#     #import os.path
#     #file_exists = os.path.exists('parameter_search.txt')
#     # if file_exists == False:
#     #     with open('parameter_search.txt', 'w') as f:
#     #         f.write('Create a new text file!')

#     with open('parameter_search.txt', 'a+') as f:
#         for line in lines:
#             f.write(line)
#         f.write('\n')
    
# except Exception as e:
#     print(e)


outlier_99
done

starting z_score
10.1
10.2
10.3
10.4
dropped 41 / 53 rows
replaced 0 / 53 rows
dropped 41 / 53 rows
replaced 0 / 53 rows
dropped 40 / 52 rows
replaced 0 / 52 rows
dropped 38 / 50 rows
replaced 0 / 50 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
dropped 40 / 40 rows
replaced 0 / 40 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
Model: LinearRegression() Combination: ('z_score_10.1',) Features: 24
Model: LinearRegression() Combination: ('z_score_10.1',) Features: 24
Lowest mae:              combo       mae     score  \
0  (z_score_10.1,)  165149.5  0.525593   

                                            features  
0  [id, date, bedrooms, 

### KNN

In [13]:
try:
    # Outlier Detection
    outlier_dict_all = {}

    if OUTLIER99 == True:
        print("outlier_99")
        outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
        df = drop99_all(df, outlier_list_99 )
        #outlier_dict_all["99_mean"] = outlier_list_99
        #outlier_dict_all["99_drop"] = outlier_list_99
        print("done\n")


    print('done \n\nstarting knn')
    for k in OUTLIER_KNN["K"]:
        for o in OUTLIER_KNN["X"]:
            print(k, o)
            outlier_list_knn = outliers_knn(df, k, o, SPLIT_SIZE)
            outlier_dict_all["knn_"+str(k)+"_"+str(o)] = outlier_list_knn

    ''''''
    outlier_lists_all = list(outlier_dict_all.values())
    outlier_lists_keys = list(outlier_dict_all.keys())
    combination_list = []
    for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
        if not i == 1: #emtpy tuple
            combination_list.append(combo)
            # print('combo #{}: {}'.format(i, combo))
    #print("...")
    #for c in combination_list[10:]:
        #print(bold("Combination: "), blue(c),"\n",)
    #print(bold(len(combination_list)), " total Combinations")

    model_obj_list = []
    for i, c in enumerate(combination_list):
        #print(f'\n{bold(i)}: {blue(c)}')
        outlier_list = get_unique_list(outlier_dict_all, c)
        try: 
            if "99_mean" in c:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
            else:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")

            #TODO Regressionmodels
            model = LinearRegression()
            obj = Model(model, (X_train, X_test, y_train, y_test), c, df)
            model_obj_list.append(obj)

        except Exception as e:
            print(bold("Error"), red(e))

    #Summary
    df_summary = pd.DataFrame(columns=['combo', 'mae', 'score'])
    for o in model_obj_list:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        row = {'combo': comb, 'mae': mae, 'score': score}
        df_summary = df_summary.append(row, ignore_index=True)

    df_summary.sort_values("mae").head()

    ##row
    mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
    score_best = df_summary[df_summary.score==df_summary.score.max()]
    best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
    print(best_model_obj)
    #print(f'{bold("Lowest mae:")} {mae_best}')

    model_obj_list_best = [best_model_obj]
    df_feature_reduction = best_model_obj.get_df().head()

    # print(df_feature_reduction)

    ###Feature
    for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
        #print(threshold, type(threshold))
        try: 
            rel_features = getRelFeatures(df_feature_reduction, threshold)
            #print(len(rel_features), rel_features)
            df_feature_reduction = df_feature_reduction[rel_features]
            X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
            best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction)
            # best_model_obj_new.train()
            model_obj_list_best.append(best_model_obj_new)

        except Exception as e:
            print(bold("Error"), red(e))
    #print(model_obj_list_best)

    #Summary
    df_summary_best = pd.DataFrame(columns=['combo', 'mae', 'score', 'features'])
    for o in model_obj_list_best:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        features = o.get_df().columns.to_list()
        features.remove("price")
        row = {'combo': comb, 'mae': mae, 'score': score, 'features': features}
        df_summary_best = df_summary_best.append(row, ignore_index=True)

    #df_summary_best.sort_values("mae")

    #row
    mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
    score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
    best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
    print(best_model_obj)
    print(f'{bold("Lowest mae:")} {mae_best}')

    lines = f'{df_summary_best["combo"][0]} - {df_summary_best["mae"][0]} - {len(df_summary_best["features"][0])} ' 

    with open('parameter_search.txt', 'a+') as f:
        for line in lines:
            f.write(line)
        f.write('\n')
    
except Exception as e:
    print(e)


outlier_99
done

done 

starting knn
3 181
3 300
3 500
3 700
3 1000
3 2000
dropped 181 / 181 rows
replaced 0 / 181 rows
dropped 300 / 300 rows
replaced 0 / 300 rows
dropped 500 / 500 rows
replaced 0 / 500 rows
dropped 700 / 700 rows
replaced 0 / 700 rows
dropped 1000 / 1000 rows
replaced 0 / 1000 rows
dropped 2000 / 2000 rows
replaced 0 / 2000 rows
dropped 300 / 300 rows
replaced 0 / 300 rows
dropped 500 / 500 rows
replaced 0 / 500 rows
dropped 700 / 700 rows
replaced 0 / 700 rows
dropped 1000 / 1000 rows
replaced 0 / 1000 rows
dropped 2000 / 2000 rows
replaced 0 / 2000 rows
dropped 500 / 500 rows
replaced 0 / 500 rows
dropped 700 / 700 rows
replaced 0 / 700 rows
dropped 1000 / 1000 rows
replaced 0 / 1000 rows
dropped 2000 / 2000 rows
replaced 0 / 2000 rows
dropped 700 / 700 rows
replaced 0 / 700 rows
dropped 1000 / 1000 rows
replaced 0 / 1000 rows
dropped 2000 / 2000 rows
replaced 0 / 2000 rows
dropped 1000 / 1000 rows
replaced 0 / 1000 rows
dropped 2000 / 2000 rows
replaced 0 / 2000 

### DB_SCAN

In [14]:
try:
    # Outlier Detection
    outlier_dict_all = {}

    if OUTLIER99 == True:
        print("outlier_99")
        outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
        df = drop99_all(df, outlier_list_99 )
        #outlier_dict_all["99_mean"] = outlier_list_99
        #outlier_dict_all["99_drop"] = outlier_list_99
        print("done\n")


    print('done \n\nstarting db_scan')
    for k in OUTLIER_DBSCAN["K"]:
        for o in OUTLIER_DBSCAN["X"]:
            for e in OUTLIER_DBSCAN["EPS"]:
                for m in OUTLIER_DBSCAN["MIN_SAMPLES"]:
                    print(k, o, e, m)
                    outlier_list_dbscan = outliers_dbscan(df, k, o, e, m, SPLIT_SIZE)
                    outlier_dict_all["db_scan"+str(k)+"_"+str(o)+str(e)+"_"+str(m)] = outlier_list_dbscan


    ''''''
    outlier_lists_all = list(outlier_dict_all.values())
    outlier_lists_keys = list(outlier_dict_all.keys())
    combination_list = []
    for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
        if not i == 1: #emtpy tuple
            combination_list.append(combo)
            # print('combo #{}: {}'.format(i, combo))
    #print("...")
    #for c in combination_list[10:]:
        #print(bold("Combination: "), blue(c),"\n",)
    #print(bold(len(combination_list)), " total Combinations")

    model_obj_list = []
    for i, c in enumerate(combination_list):
        #print(f'\n{bold(i)}: {blue(c)}')
        outlier_list = get_unique_list(outlier_dict_all, c)
        try: 
            if "99_mean" in c:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
            else:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")

            #TODO Regressionmodels
            model = LinearRegression()
            obj = Model(model, (X_train, X_test, y_train, y_test), c, df)
            model_obj_list.append(obj)

        except Exception as e:
            print(bold("Error"), red(e))

    #Summary
    df_summary = pd.DataFrame(columns=['combo', 'mae', 'score'])
    for o in model_obj_list:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        row = {'combo': comb, 'mae': mae, 'score': score}
        df_summary = df_summary.append(row, ignore_index=True)

    df_summary.sort_values("mae").head()

    ##row
    mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
    score_best = df_summary[df_summary.score==df_summary.score.max()]
    best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
    print(best_model_obj)
    #print(f'{bold("Lowest mae:")} {mae_best}')

    model_obj_list_best = [best_model_obj]
    df_feature_reduction = best_model_obj.get_df().head()

    # print(df_feature_reduction)

    ###Feature
    for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
        #print(threshold, type(threshold))
        try: 
            rel_features = getRelFeatures(df_feature_reduction, threshold)
            #print(len(rel_features), rel_features)
            df_feature_reduction = df_feature_reduction[rel_features]
            X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
            best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction)
            # best_model_obj_new.train()
            model_obj_list_best.append(best_model_obj_new)

        except Exception as e:
            print(bold("Error"), red(e))
    #print(model_obj_list_best)

    #Summary
    df_summary_best = pd.DataFrame(columns=['combo', 'mae', 'score', 'features'])
    for o in model_obj_list_best:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        features = o.get_df().columns.to_list()
        features.remove("price")
        row = {'combo': comb, 'mae': mae, 'score': score, 'features': features}
        df_summary_best = df_summary_best.append(row, ignore_index=True)

    #df_summary_best.sort_values("mae")

    #row
    mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
    score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
    best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
    print(best_model_obj)
    print(f'{bold("Lowest mae:")} {mae_best}')

    lines = f'{df_summary_best["combo"][0]} - {df_summary_best["mae"][0]} - {len(df_summary_best["features"][0])} ' 

    with open('parameter_search.txt', 'a+') as f:
        for line in lines:
            f.write(line)
        f.write('\n')
    
except Exception as e:
    print(e)


outlier_99
done

done 

starting db_scan
3 181 0.3 3
3 181 0.3 9
3 181 0.3 10
3 181 0.7 3
3 181 0.7 9
3 181 0.7 10
3 2000 0.3 3
3 2000 0.3 9
3 2000 0.3 10
3 2000 0.7 3
3 2000 0.7 9
3 2000 0.7 10
dropped 170 / 170 rows
replaced 0 / 170 rows
Model: LinearRegression() Combination: ('db_scan',) Features: 24
Model: LinearRegression() Combination: ('db_scan',) Features: 24
Lowest mae:         combo        mae     score  \
0  (db_scan,)  162674.96  0.482159   

                                            features  
0  [id, date, bedrooms, bathrooms, sqft_living, s...  


### Noise

In [15]:
try:
    # Outlier Detection
    outlier_dict_all = {}

    if OUTLIER99 == True:
        print("outlier_99")
        outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
        df = drop99_all(df, outlier_list_99 )
        #outlier_dict_all["99_mean"] = outlier_list_99
        #outlier_dict_all["99_drop"] = outlier_list_99
        print("done\n")

    print('done \n\nstarting noise')
    for cv in OUTLIER_NOISE["CV"]:
            print(cv)
            outlier_list_noise = getNoise(df, cv)
            outlier_dict_all["noise_"+str(cv)] = outlier_list_noise
    print('done')


    ''''''
    outlier_lists_all = list(outlier_dict_all.values())
    outlier_lists_keys = list(outlier_dict_all.keys())
    combination_list = []
    for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
        if not i == 1: #emtpy tuple
            combination_list.append(combo)
            # print('combo #{}: {}'.format(i, combo))
    #print("...")
    #for c in combination_list[10:]:
        #print(bold("Combination: "), blue(c),"\n",)
    #print(bold(len(combination_list)), " total Combinations")

    model_obj_list = []
    for i, c in enumerate(combination_list):
        #print(f'\n{bold(i)}: {blue(c)}')
        outlier_list = get_unique_list(outlier_dict_all, c)
        try: 
            if "99_mean" in c:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
            else:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")

            #TODO Regressionmodels
            model = LinearRegression()
            obj = Model(model, (X_train, X_test, y_train, y_test), c, df)
            model_obj_list.append(obj)

        except Exception as e:
            print(bold("Error"), red(e))

    #Summary
    df_summary = pd.DataFrame(columns=['combo', 'mae', 'score'])
    for o in model_obj_list:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        row = {'combo': comb, 'mae': mae, 'score': score}
        df_summary = df_summary.append(row, ignore_index=True)

    df_summary.sort_values("mae").head()

    ##row
    mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
    score_best = df_summary[df_summary.score==df_summary.score.max()]
    best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
    print(best_model_obj)
    #print(f'{bold("Lowest mae:")} {mae_best}')

    model_obj_list_best = [best_model_obj]
    df_feature_reduction = best_model_obj.get_df().head()

    # print(df_feature_reduction)

    ###Feature
    for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
        #print(threshold, type(threshold))
        try: 
            rel_features = getRelFeatures(df_feature_reduction, threshold)
            #print(len(rel_features), rel_features)
            df_feature_reduction = df_feature_reduction[rel_features]
            X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
            best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction)
            # best_model_obj_new.train()
            model_obj_list_best.append(best_model_obj_new)

        except Exception as e:
            print(bold("Error"), red(e))
    #print(model_obj_list_best)

    #Summary
    df_summary_best = pd.DataFrame(columns=['combo', 'mae', 'score', 'features'])
    for o in model_obj_list_best:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        features = o.get_df().columns.to_list()
        features.remove("price")
        row = {'combo': comb, 'mae': mae, 'score': score, 'features': features}
        df_summary_best = df_summary_best.append(row, ignore_index=True)

    #df_summary_best.sort_values("mae")

    #row
    mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
    score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
    best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
    print(best_model_obj)
    print(f'{bold("Lowest mae:")} {mae_best}')

    lines = f'{df_summary_best["combo"][0]} - {df_summary_best["mae"][0]} - {len(df_summary_best["features"][0])} ' 

    #import os.path
    #file_exists = os.path.exists('parameter_search.txt')
    # if file_exists == False:
    #     with open('parameter_search.txt', 'w') as f:
    #         f.write('Create a new text file!')

    with open('parameter_search.txt', 'a+') as f:
        for line in lines:
            f.write(line)
        f.write('\n')
    
except Exception as e:
    print(e)


outlier_99
done

done 

starting noise
3
4
5
6
7
8
9
10
done
dropped 35 / 51 rows
replaced 0 / 51 rows
dropped 36 / 51 rows
replaced 0 / 51 rows
dropped 35 / 51 rows
replaced 0 / 51 rows
dropped 40 / 56 rows
replaced 0 / 56 rows
dropped 37 / 54 rows
replaced 0 / 54 rows
dropped 38 / 56 rows
replaced 0 / 56 rows
dropped 40 / 58 rows
replaced 0 / 58 rows
dropped 39 / 59 rows
replaced 0 / 59 rows
dropped 40 / 40 rows
replaced 0 / 40 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
dropped 43 / 43 rows
replaced 0 / 43 rows
dropped 42 / 42 rows
replaced 0 / 42 rows
dropped 41 / 41 rows
replaced 0 / 41 rows
dropped 44 / 44 rows
replaced 0 / 44 rows
dropped 43 / 43 rows
replaced 0 / 43 rows
dropped 42 / 42 rows
replaced 0 / 42 rows
dropped 45 / 45 rows
replaced 0 / 45 rows
dropped 40 / 40 rows
replaced 0 / 40 rows
dropped 42 / 42 rows
replaced 0 / 42 rows
dropped 43 / 43 rows
replaced 0 / 43 rows
dropped 43 / 43 rows
replaced 0 / 43 rows
dropped 42 / 42 rows
replaced 0 / 42 rows
dropped 39 / 39